In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
df = pd.read_excel('/content/drive/MyDrive/Nhóm 7/Đồ án môn học/DATA_TRAINING/final_dataset_before_quality_control.xlsx')
df.head()

,mssv,namsinh,gioitinh,khoa,hedt,khoahoc,dtb_toankhoa,dtb_tichluy,sotc_tichluy,diemtbhk_1,...,diemtbhk_10,diemtbhk_11,diemtbhk_12,diemtbhk_13,diemtbhk_14,diemtbhk_15,diemtbhk_16,diemtbhk_17,diemtbhk_18,diemtbhk_19
0,0000AC05XPvAibaEXe9B2tolTZ0JLoBGbkQixQS6,NaN,NaN,NaN,NaN,NaN,7.91,7.91,62.0,7.79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0001CF3EXPvAibaEXe/xTpnJlY3K6L35F+TKUux6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,2001.0,0.0,HTTT,CQUI,14.0,8.79,8.79,129.0,8.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD,1995.0,1.0,KTMT,CQUI,8.0,7.26,7.26,147.0,5.41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0006A0BBXPvAibaEXe/lMOwHQdw54DgUkWaqwb1u,NaN,NaN,NaN,NaN,NaN,7.27,7.27,26.0,7.27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
import pandas as pd
import numpy as np

df = df.drop(['khoa', 'hedt', 'khoahoc', 'diemtbhk_13', 'diemtbhk_14', 'diemtbhk_15', 'diemtbhk_16', 'diemtbhk_17', 'diemtbhk_18', 'diemtbhk_19'], axis=1)
mean_nam_sinh = int(df['namsinh'].mean(skipna=True))
df['namsinh'] = df['namsinh'].fillna(mean_nam_sinh)
mode_value = df['gioitinh'].mode()[0]
df['gioitinh'] = df['gioitinh'].fillna(mode_value)
df.head()

all_columns = df.columns.tolist()
exclude_columns = ['mssv', 'namsinh', 'gioitinh']
columns_to_fill = [col for col in all_columns if col not in exclude_columns]
df[columns_to_fill] = df[columns_to_fill].fillna(0)
df.head()

,mssv,namsinh,gioitinh,dtb_toankhoa,dtb_tichluy,sotc_tichluy,diemtbhk_1,diemtbhk_2,diemtbhk_3,diemtbhk_4,diemtbhk_5,diemtbhk_6,diemtbhk_7,diemtbhk_8,diemtbhk_9,diemtbhk_10,diemtbhk_11,diemtbhk_12
0,0000AC05XPvAibaEXe9B2tolTZ0JLoBGbkQixQS6,1998.0,1.0,7.91,7.91,62.0,7.79,8.28,7.56,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
1,0001CF3EXPvAibaEXe/xTpnJlY3K6L35F+TKUux6,1998.0,1.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
2,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,2001.0,0.0,8.79,8.79,129.0,8.84,9.00,9.11,8.75,8.54,8.74,8.41,0.0,0.0,0.0,0.0,0.0
3,00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD,1995.0,1.0,7.26,7.26,147.0,5.41,7.07,6.36,8.32,7.86,8.09,6.92,7.8,8.2,0.0,0.0,0.0
4,0006A0BBXPvAibaEXe/lMOwHQdw54DgUkWaqwb1u,1998.0,1.0,7.27,7.27,26.0,7.27,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0


In [8]:

# Hàm xây dựng mô hình Phased LSTM
def build_phased_lstm(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(50, activation='tanh', recurrent_activation='sigmoid'),
        Dense(1)  # Đầu ra là một giá trị
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

# Độ đo
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100 if np.all(y_true != 0) else np.nan
    mpe = np.mean((y_true - y_pred) / y_true) * 100 if np.all(y_true != 0) else np.nan
    r2 = r2_score(y_true, y_pred)
    return {"R2 Score": r2, "MSE": mse, "RMSE": rmse, "MAE": mae, "MAPE": mape, "MPE": mpe}

# Cột ban đầu cho X
initial_columns = ['namsinh', 'gioitinh', 'dtb_toankhoa', 'dtb_tichluy', 'sotc_tichluy', 'diemtbhk_1']

# Lặp qua từng cột làm y
results = []
current_X_columns = initial_columns.copy()

for i in range(len(initial_columns), len(df.columns) - 1):  # Bắt đầu từ cột tiếp theo sau `initial_columns`
    target_column = df.columns[i]  # Cột y hiện tại
    next_target_column = df.columns[i + 1]  # Cột y mới (vòng tiếp theo)

    # Xác định X và y
    X = df[current_X_columns]
    y = df[target_column]

    # Chia dữ liệu
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    # Chuẩn hóa dữ liệu cho Phased LSTM
    X_train_lstm = np.expand_dims(X_train.to_numpy(), axis=-1)  # Thêm chiều
    X_val_lstm = np.expand_dims(X_val.to_numpy(), axis=-1)
    X_test_lstm = np.expand_dims(X_test.to_numpy(), axis=-1)

    # Xây dựng và huấn luyện mô hình Phased LSTM
    lstm_model = build_phased_lstm(input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]))
    lstm_model.fit(X_train_lstm, y_train, validation_data=(X_val_lstm, y_val), epochs=10, batch_size=32, verbose=1)

    # Dự đoán và tính toán độ đo
    y_pred = lstm_model.predict(X_test_lstm).flatten()
    metrics = calculate_metrics(y_test, y_pred)
    metrics["Target Column"] = target_column
    results.append(metrics)

    # Cập nhật X cho vòng tiếp theo
    current_X_columns.append(target_column)

# Tạo DataFrame kết quả
results_df = pd.DataFrame(results)

# Hiển thị kết quả
print(results_df)

# Lưu kết quả ra file CSV
results_df.to_csv("phased_lstm_results.csv", index=False)


Epoch 1/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 16.9395 - mae: 2.8373 - val_loss: 0.2906 - val_mae: 0.3094
Epoch 2/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.2241 - mae: 0.2514 - val_loss: 0.0841 - val_mae: 0.1848
Epoch 3/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0638 - mae: 0.1311 - val_loss: 0.0373 - val_mae: 0.0951
Epoch 4/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0272 - mae: 0.0836 - val_loss: 0.0229 - val_mae: 0.0809
Epoch 5/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0206 - mae: 0.0697 - val_loss: 0.0141 - val_mae: 0.0641
Epoch 6/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0095 - mae: 0.0482 - val_loss: 0.0090 - val_mae: 0.0608
Epoch 7/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0051 - mae: 0.0396 - val_loss: 0.0036 - val_mae: 0.0305
Epoch 8/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0026 - mae: 0.0279 - val_loss: 0.0025 - val_mae: 0.0300
Epoch 9/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - l

In [13]:
# Tính mean cho tất cả các cột số
mean_values = results_df.mean()
print(mean_values)


TypeError: Could not convert ['diemtbhk_1diemtbhk_2diemtbhk_3diemtbhk_4diemtbhk_5diemtbhk_6diemtbhk_7diemtbhk_8diemtbhk_9diemtbhk_10diemtbhk_11'] to numeric

In [16]:
results_df

,R2 Score,MSE,RMSE,MAE,MAPE,MPE,Target Column
0,0.999905,0.001128,0.033590,0.016123,NaN,NaN,diemtbhk_1
1,0.924091,0.952229,0.975822,0.524431,NaN,NaN,diemtbhk_2
2,0.951350,0.638490,0.799056,0.411377,NaN,NaN,diemtbhk_3
3,0.931202,0.910943,0.954433,0.464973,NaN,NaN,diemtbhk_4
4,0.960994,0.526169,0.725375,0.380553,NaN,NaN,diemtbhk_5
5,0.939924,0.776440,0.881159,0.402179,NaN,NaN,diemtbhk_6
6,0.945729,0.713175,0.844497,0.383136,NaN,NaN,diemtbhk_7
7,0.756204,2.599799,1.612389,0.707039,NaN,NaN,diemtbhk_8
8,0.510294,3.383896,1.839537,0.891083,NaN,NaN,diemtbhk_9
9,0.352101,1.122826,1.059635,0.364422,NaN,NaN,diemtbhk_10


In [19]:
print(results_df['R2 Score'].mean())
print(results_df['MSE'].mean())
print(results_df['RMSE'].mean())
print(results_df['MAE'].mean())

0.778732503236526
1.1175196911805323
0.9584157916505505
0.4320264782371889
